In [3]:
import logging
from os import chdir

import pandas as pd
from pandas import Series

from unicef_schools_attribute_cleaning.models.School import School
from unicef_schools_attribute_cleaning.models.Connectivity import Connectivity

from unicef_schools_attribute_cleaning.utils.standardize_column_names import standardize_column_names

chdir('/Users/alex/repos/unicef-schools-attribute-cleaning')
df = pd.read_excel('./data/UNICE_schools_raw_2020_Jun/Zimbabwe schools complete data.xlsx')

# apply country code, provider to every row
df['country_code'] = 'ZW'
df['owner'] = 'UNICEF'
df['is_private'] = True
df['provider'] = 'unknown'
df['provider_is_private'] = True

df2 = standardize_column_names(df)
df2['lat']
# connectivity_values = df2['type_connectivity'].unique()
# logging.warning(connectivity_values)

# def series_to_school(row: Series) -> School:
#     return School.parse_obj(row.to_dict())

# df2.apply(func=series_to_school, axis=1)



0      -17.8264
1      -17.7915
2      -17.7881
3      -17.8767
4      -17.7917
         ...   
9620    (blank)
9621    (blank)
9622    (blank)
9623    (blank)
9624    (blank)
Name: lat, Length: 9625, dtype: object